In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

import random
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [2]:
n_samples = 1000

year = np.random.randint(2016, 2021, n_samples) #год выпуска фильма
runtime =  np.random.randint(60, 180, n_samples) #продолжительность фильма
viewers = np.random.choice(200, n_samples) #количество просмотров в миллионах
average_ticket_price = np.random.randint(10, 30, n_samples) #средняя стоимость просмотра в кинотеатре

box_office = viewers * average_ticket_price # пусть рейтинг зависит не от стоимости билета напрямую, а от суммарной прибыли за фильм

ratings = runtime // 5 + box_office // 25 #пусть рейтинг фильма определяется по формуле, причем от года выпуска оценка зависеть не будет

data = pd.DataFrame({'year': year,'runtime': runtime,'viewers, mln': viewers, 'average_ticket_price, $': average_ticket_price, 'ratings':ratings})
data.head()

,year,runtime,"viewers, mln","average_ticket_price, $",ratings
0,2020,106,185,16,139
1,2019,70,144,11,77
2,2017,142,44,17,57
3,2017,141,72,29,111
4,2017,138,9,15,32


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype
---  ------                   --------------  -----
 0   year                     1000 non-null   int32
 1   runtime                  1000 non-null   int32
 2   viewers, mln             1000 non-null   int32
 3   average_ticket_price, $  1000 non-null   int32
 4   ratings                  1000 non-null   int32
dtypes: int32(5)
memory usage: 19.7 KB


#### Попробуем построить модель на основе всех признаков

In [30]:
X = data[['year', 'runtime', 'viewers, mln',  'average_ticket_price, $']]
y = data['ratings']

reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))
pred_values = reg.predict(data[['year', 'runtime', 'viewers, mln',  'average_ticket_price, $']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [0.33137434 0.19485299 0.77966118 3.92135994]
Bias: -744.8213259806247
Error: 9.771368764229978


#### Введем новый признак Box-office и заменим им в модели признаки средней стоимости и количества зрителей.

In [31]:
data['box_office'] = data['viewers, mln'] * data['average_ticket_price, $']
data.head(5)

,year,runtime,"viewers, mln","average_ticket_price, $",ratings,box_office
0,2020,146,140,17,124,2380
1,2019,104,53,24,70,1272
2,2017,137,97,17,92,1649
3,2016,83,29,27,47,783
4,2020,117,165,15,122,2475


In [32]:
X = data[['year', 'runtime', 'box_office']]
y = data['ratings']

reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))
pred_values = reg.predict(data[['year', 'runtime', 'box_office']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [-0.00390497  0.19946187  0.03998821]
Bias: 7.144828358358708
Error: 0.33622354123395554


#### Отбросим первый признак с наименьшим весом

In [33]:
X = data[['runtime', 'box_office']]
y = data['ratings']
reg = LinearRegression().fit(X, y)

print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))
pred_values = reg.predict(data[['runtime', 'box_office']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [0.19945694 0.03998826]
Bias: -0.7350490792802447
Error: 0.33635049220170526
